# Building a simple machine learning program on Serie A 2018/19 Season Stats - Using TensorFlow

Let us begin by importing the necessary packages.

In [1]:
import numpy as np
import pandas as pd

Defining a general function that will take a dataset of football/soccer results and output the number of wins, draws, and losses. Note that the 'data' should be in Panda DataFrame format, with FTR being a category/heading of a column. In our datasets, this is a given.

In [2]:
# General function that isolates the full time result to see record of wins/losses/draws

def winRecords(data, leagueName):
    
    ftr = data['FTR']

    homeWins=0
    awayWins=0
    draws=0
    
    for i in range(0, len(ftr.to_numpy())-1):
    
        if (ftr.to_numpy()[i] == 'H'):
            homeWins += 1
            #print(j)
        elif (ftr.to_numpy()[i] == 'A'):
            awayWins += 1
        else:
            draws +=1

    return leagueName, 'HomeWins: %s' % homeWins, 'Away Wins: %s' % awayWins, 'Draws: %s' % draws

Now, let's read the files and print out the stats for Serie A leagues in several seasons (and the EPL season).

In [3]:
print(winRecords(pd.read_csv("serieA_season-1516.csv"), 'SerieA_15_16'))
print(winRecords(pd.read_csv("serieA_season-1617.csv"), 'SerieA_16_17'))
print(winRecords(pd.read_csv("serieA_season-1819.csv"), 'SerieA_18_19'))
print(winRecords(pd.read_csv("epl_season-1819.csv"), 'EPL_18_19'))

('SerieA_15_16', 'HomeWins: 175', 'Away Wins: 109', 'Draws: 95')
('SerieA_16_17', 'HomeWins: 183', 'Away Wins: 116', 'Draws: 80')
('SerieA_18_19', 'HomeWins: 165', 'Away Wins: 106', 'Draws: 108')
('EPL_18_19', 'HomeWins: 181', 'Away Wins: 127', 'Draws: 71')


# Let's start training/learning/evaluating/testing our Model

Beginning with Serie A 2018/19 dataset, we must first 'clean' the data. This is done by dropping the useless categories. In our case, this is the division and the date of the games. We don't care when the games were played (they're already ordered chronologically), but who played whom and when. We also want to drop the Full Time Home and Away Goals Scored and the Half Time Home & Away Goals and Result categories, as these will obviously influence our data.

In [4]:
dataSerieA1819 = pd.read_csv("serieA_season-1819.csv")

# Dropping the Div and Dates

dataSerieA1819 = dataSerieA1819.drop('Div', axis=1)
dataSerieA1819 = dataSerieA1819.drop('Date', axis=1)

# Dropping the Full Time Home Goals Scored, FTAG

dataSerieA1819 = dataSerieA1819.drop('FTAG', axis=1)
dataSerieA1819 = dataSerieA1819.drop('FTHG', axis=1)

# Dropping the Half Time Home Goals Scored, HTAG, and the Half time result

dataSerieA1819 = dataSerieA1819.drop('HTHG', axis=1)
dataSerieA1819 = dataSerieA1819.drop('HTAG', axis=1)
dataSerieA1819 = dataSerieA1819.drop('HTR', axis=1)

dataSerieA1819['FTR'].unique()

array(['A', 'H', 'D'], dtype=object)

We will take a look at the column of interest, which is FTR (Full Time Result), and change it to 0s and 1s for our system to be able to classify/predict these outcomes easily. 0 if the Home Team draws or loses, 1 if the Home Team wins.

In [5]:
# Change the string for Full Time Result (FTR), AKA the outcome to 0s and 1s. 1 if the Home Team wins, 0 if the Away Team won or if a Draw was achieved

def fix_outcome(outcome):
    if outcome == 'H':
        return 1
    else:
        return 0

dataSerieA1819['FTR'] = dataSerieA1819['FTR'].apply(fix_outcome)

Import the train_test_split function from the sklearn package.

In [6]:
from sklearn.model_selection import train_test_split

Now we can Train_Test_Split our data

In [7]:
# Train Test Split Data

x_data = dataSerieA1819.drop('FTR', axis=1)
y_labels = dataSerieA1819['FTR']
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

Import TensorFlow package

In [8]:
import tensorflow as tf

Create the feature columns. In this notebook, this is split up into a section for the categorical values, and a section for the continuous/numerical values. This is because they use different lines of code.

In [9]:
# Create tf.feature_columns for Categorical Values

HomeTeam = tf.feature_column.categorical_column_with_hash_bucket('HomeTeam', hash_bucket_size=1000)
AwayTeam = tf.feature_column.categorical_column_with_hash_bucket('AwayTeam', hash_bucket_size=1000)

Note that for these values, there is not an 'FTR' column.

In [10]:
# Create tf.feature_columns for Numerical/Continuous Values

HS   = tf.feature_column.numeric_column('HS')   # Home Shots
AS   = tf.feature_column.numeric_column('AS')   # Away Shots
HST  = tf.feature_column.numeric_column('HST')  # Home Shots on Target
AST  = tf.feature_column.numeric_column('AST')  # Away Shots on Target
HF  = tf.feature_column.numeric_column('HF')    # Home Fouls
AF  = tf.feature_column.numeric_column('AF')    # Away Fouls
HC  = tf.feature_column.numeric_column('HC')    # Home Corners
AC  = tf.feature_column.numeric_column('AC')    # Away Corners
HY  = tf.feature_column.numeric_column('HY')    # Home Yellow Cards
AY  = tf.feature_column.numeric_column('AY')    # Away Yellow Cards
HR  = tf.feature_column.numeric_column('HR')    # Home Red Cards
AR  = tf.feature_column.numeric_column('AR')    # Away Red Cards

Compile into one features column.

In [11]:
# Compile into one features column

feature_cols = [HomeTeam,AwayTeam,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR]

Using the tensor flow estimator method, let's build our input function. Note that the batch_size is 100 and the number of epochs is "None". The batch size is the number of samples that are processed before updating the model, while the epochs are the number of complete passes through the training dataset.

In [12]:
# Building an input function

input_fnc = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

Using TensorFlow Estimator, with Linear Classifier, let's create our model.

In [13]:
# Using Linear Classifier to create a model with tf.Estimator

model = tf.estimator.LinearClassifier(feature_columns=feature_cols)

model.train(input_fn=input_fnc, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12c98fb10>, '_model_dir': '/var/folders/tw/j24f0q85731142mf21hrd98c0000gn/T/tmpH1UCp1', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
Instructions for updating:
Use Variable.read_value. Varia

INFO:tensorflow:loss = 46.10028, step = 2601 (0.783 sec)
INFO:tensorflow:global_step/sec: 163.285
INFO:tensorflow:loss = 44.507694, step = 2701 (0.613 sec)
INFO:tensorflow:global_step/sec: 142.915
INFO:tensorflow:loss = 42.481216, step = 2801 (0.698 sec)
INFO:tensorflow:global_step/sec: 165.608
INFO:tensorflow:loss = 38.050945, step = 2901 (0.607 sec)
INFO:tensorflow:global_step/sec: 144.336
INFO:tensorflow:loss = 52.51755, step = 3001 (0.689 sec)
INFO:tensorflow:global_step/sec: 157.025
INFO:tensorflow:loss = 34.82693, step = 3101 (0.643 sec)
INFO:tensorflow:global_step/sec: 140.239
INFO:tensorflow:loss = 40.723103, step = 3201 (0.707 sec)
INFO:tensorflow:global_step/sec: 158.891
INFO:tensorflow:loss = 44.055832, step = 3301 (0.629 sec)
INFO:tensorflow:global_step/sec: 142.533
INFO:tensorflow:loss = 49.15535, step = 3401 (0.705 sec)
INFO:tensorflow:global_step/sec: 168.745
INFO:tensorflow:loss = 46.509056, step = 3501 (0.593 sec)
INFO:tensorflow:global_step/sec: 161.452
INFO:tensorflo

Using model.predict and a new input function pred_fn, generate predictions. Use list() to put those predictions in a list.

In [14]:
# Now to make 'predictions'

pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size = len(X_test), shuffle=False)
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/tw/j24f0q85731142mf21hrd98c0000gn/T/tmpH1UCp1/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


What do the predictions look like? 

In [15]:
print(predictions[0])

{'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array(['0', '1'], dtype=object), 'probabilities': array([0.64962864, 0.35037133], dtype=float32), 'classes': array(['0'], dtype=object), 'logistic': array([0.35037136], dtype=float32), 'logits': array([-0.61740726], dtype=float32), 'class_ids': array([0])}


That's interesting. Each prediction has a logit, logistic, probability, class_id, etc. associated with it. We care most about the class_ids as that will indicate whether the Home Team won the match or not. In this model, draws are considered losses. Iterate over the list of predictions, appending only the relevant category, class_ids, to a new list.

In [16]:
final_preds= []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [17]:
print(final_preds[:8])

[0, 1, 0, 0, 1, 0, 0, 1]


Time to evaluate, using sklearn metrics, how accurate our model actually is! Import Classification_report from the sklearn metrics package.

In [18]:
from sklearn.metrics import classification_report

In [19]:
# Import Classification report from SkLearn-Metrics to evaluate the performance of the test model

print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.65      0.70      0.68        61
           1       0.62      0.57      0.59        53

   micro avg       0.64      0.64      0.64       114
   macro avg       0.64      0.64      0.64       114
weighted avg       0.64      0.64      0.64       114



Cleaning up the code, I create separate functions for each process. cleanData drops the irrelevant columns and/or rows.

In [20]:
# Let's start with the training/learning/evaluating/testing Machine Learning Method! First, we need to clean the data.
# Note: This script was written assuming all the data files will have the same default columns. This is true for the datasets that I've downloaded from the internet.
# I removed (in some of these files) the betting odds, as I figure these may influence results in ways outside of the determinants of the game. 
# Odds are set by bookkeepers, based on their own analysis.


def cleanData(dataFileCSV):

    data = pd.read_csv(dataFileCSV)

    # Dropping the Div and Dates

    data = data.drop('Div', axis=1)
    data = data.drop('Date', axis=1)

    # Dropping the Full Time Home Goals Scored, FTAG

    data = data.drop('FTAG', axis=1)
    data = data.drop('FTHG', axis=1)

    # Dropping the Half Time Home Goals Scored, HTAG, and the Half time result

    data = data.drop('HTHG', axis=1)
    data = data.drop('HTAG', axis=1)
    data = data.drop('HTR', axis=1)

    data['FTR'].unique()

    return data

The _fix outcome_ function stays the same as above, so I won't include it here. _trainModel_ takes in clean data, creates and trains a model. This function returns a _trained model_.

In [21]:
def trainModel(data):
    data['FTR'] = data['FTR'].apply(fix_outcome)

    # Train Test Split Data

    x_data = data.drop('FTR', axis=1)
    y_labels = data['FTR']
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)


    # Create tf.feature_columns for Categorical Values

    HomeTeam = tf.feature_column.categorical_column_with_hash_bucket('HomeTeam', hash_bucket_size=1000)
    AwayTeam = tf.feature_column.categorical_column_with_hash_bucket('AwayTeam', hash_bucket_size=1000)
    

    # Create tf.feature_columns for Numerical/Continuous Values

    HS   = tf.feature_column.numeric_column('HS')   # Home Shots
    AS   = tf.feature_column.numeric_column('AS')   # Away Shots
    HST  = tf.feature_column.numeric_column('HST')  # Home Shots on Target
    AST  = tf.feature_column.numeric_column('AST')  # Away Shots on Target
    HF  = tf.feature_column.numeric_column('HF')    # Home Fouls
    AF  = tf.feature_column.numeric_column('AF')    # Away Fouls
    HC  = tf.feature_column.numeric_column('HC')    # Home Corners
    AC  = tf.feature_column.numeric_column('AC')    # Away Corners
    HY  = tf.feature_column.numeric_column('HY')    # Home Yellow Cards
    AY  = tf.feature_column.numeric_column('AY')    # Away Yellow Cards
    HR  = tf.feature_column.numeric_column('HR')    # Home Red Cards
    AR  = tf.feature_column.numeric_column('AR')    # Away Red Cards


    # Compile into one features column
    
    feature_cols = [HomeTeam,AwayTeam,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR]

    # Building an input function

    input_fnc = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

    # Using Linear Classifier to create a model with tf.Estimator

    model = tf.estimator.LinearClassifier(feature_columns=feature_cols)

    model.train(input_fn=input_fnc, steps=5000)

    return model, X_test, y_test

Now, we define a function that makes predictions and evaluates/tests the model using sklearn-classification-report

In [22]:
# Now to make 'predictions'

def predict(data):

    model, X_test, y_test = trainModel(data)

    pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size = len(X_test), shuffle=False)
    predictions = list(model.predict(input_fn=pred_fn))

    print(predictions[0])

    final_preds= []
    for pred in predictions:
        final_preds.append(pred['class_ids'][0])

    print(final_preds[:10])

    # Import Classification report from SkLearn-Metrics to evaluate the performance of the test model

    print(classification_report(y_test, final_preds))
    return final_preds

Let's create a main() function that compiles all these other functions so that we only have to write in one function every time we want to test new data! This is fairly reminiscent of OOP principles, something I'd normally do when I'm using Java or C. I'm not sure yet whether it's better notation to just have one function and have all these sub-functions as sub-functions, or just place them as lines of code in one large main function, or to have them split up as such. For now, I'll keep them split up. At the very least, it helps break down the code and make it easier to comprehend.

In [23]:
# Compiling everything into a 'main' function - this is reminiscent of OOP principles, something I'd do in Java - but it's nice to keep things compact, even in Python

def main(data):
    return predict(cleanData(data))

# I suppose I could very easily just create one main function and wrap the other 'predict' and 'cleanData' methods into the main function, but I'm still undecided as to whether that is better documentation than splitting it up.

Using the same (serieA_season-1819.csv) data as above, let's see if we get the same results. Hint: if we return the right datatypes for each function, we should get the exact same results.

In [24]:
main('serieA_season-1819.csv')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12da22990>, '_model_dir': '/var/folders/tw/j24f0q85731142mf21hrd98c0000gn/T/tmpep1oM5', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done ca

INFO:tensorflow:global_step/sec: 170.074
INFO:tensorflow:loss = 35.524277, step = 4701 (0.586 sec)
INFO:tensorflow:global_step/sec: 154.36
INFO:tensorflow:loss = 44.550823, step = 4801 (0.647 sec)
INFO:tensorflow:global_step/sec: 128.709
INFO:tensorflow:loss = 43.118744, step = 4901 (0.782 sec)
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/tw/j24f0q85731142mf21hrd98c0000gn/T/tmpep1oM5/model.ckpt.
INFO:tensorflow:Loss for final step: 49.868595.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/tw/j24f0q85731142mf21hrd98c0000gn/T/tmpep1oM5/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array(['0', '1'], dtype=object), 'probabilities': array([0.6462353 , 0.35376468], dtype=float32), 'classes': array(['0'], dtype=object), 'logistic': array([0.3537647

[0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0]

Voila! We do! :)